# 19.Proje-Salary Prediction for Job Postings-İş İlanları Maaş Tahmini
# Regression

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
warnings.filterwarnings("ignore")
pd.set_option("display.max_columns",100)

In [2]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/usjobs/usjobs_train.csv/usjobs_train.csv
/kaggle/input/usjobs/usjobs_test.csv/usjobs_test.csv


In [3]:
dftrain=pd.read_csv("/kaggle/input/usjobs/usjobs_train.csv/usjobs_train.csv")
dftest=pd.read_csv("/kaggle/input/usjobs/usjobs_test.csv/usjobs_test.csv")

In [4]:
dftrain.head()

,ID,Job,Jobs_Group,Profile,Remote,Company,Location,City,State,Frecuency_Salary,Mean_Salary,Skills,Sector,Sector_Group,Revenue,Employee,Company_Score,Reviews,Director,Director_Score,URL
0,job_f2c807527f687b96,"Part-time Reporting Business Analyst, Data & A...",Financial Analyst,NaN,Remote,Sandy Hook Promise Foundation,Remote,NaN,NaN,hour,115000.0,"['Salesforce', 'Bachelor']",NGOs and Nonprofit Organizations,Nonprofit Organizations,NaN,XS,4.2,20.0,NaN,NaN,https://www.sandyhookpromise.org/
1,job_2660d4c53505af10,Controller,Controller,NaN,NaN,Building Service 32BJ Benefit Funds,"New York, NY 10013 (Tribeca area)",New York,NY,year,185000.0,"['SQL', 'Master', 'Dynamics 365', 'Snowflake',...",NGOs and Nonprofit Organizations,Nonprofit Organizations,NaN,M,3.5,58.0,"Peter Goldberger, Executive Director",0.70,NaN
2,sj_50358c44328ae06a,Sr Finance Analyst,Financial Analyst,Senior,NaN,LCS,NaN,NaN,NaN,year,84500.0,"['Word', 'Bachelor', 'Excel']",Personal Consumer Services,Sales,XXXS,XXXS,3.4,88.0,NaN,NaN,NaN
3,job_a087fd700e3e85f0,Senior Business Intelligence Analyst,Business Intelligence,Senior,Hybrid,Federal Reserve Bank of Richmond,"Richmond, VA 23219 (Central Office area)",Richmond,VA,year,111625.0,"['PowerPoint', 'Power BI', 'Tableau', 'Word', ...",Banking and Credit Services,Finance,XXL,XL,3.8,30.0,Tom Barkin,0.70,https://www.richmondfed.org/
4,job_d2a2538a2c4d2033,Data Center Operations Analyst (Temporary Assi...,Operations Analyst,NaN,Remote,Los Angeles County Office of Education,"Downey, CA 90242+1 ubicación",Downey,CA,hour,102690.4,['Office'],State and Regional Agencies,Government,NaN,XL,4.2,186.0,Debra Duardo,0.85,NaN


In [5]:
print(dftrain.shape)
print(dftest.shape)

(33248, 21)
(22166, 20)


In [6]:
dftest['Mean_Salary'] = 0

In [7]:
dftest.head(1)

,ID,Job,Jobs_Group,Profile,Remote,Company,Location,City,State,Frecuency_Salary,Skills,Sector,Sector_Group,Revenue,Employee,Company_Score,Reviews,Director,Director_Score,URL,Mean_Salary
0,sj_99ad4f80ae7f4835,Business Analyst Manager,Business Analyst,Lead,Remote,Dryden Mutual Insurance Company,"Dryden, NY 13053",Dryden,NY,year,"['PowerPoint', 'Office', 'Excel', 'Agile']",Insurance Companies and Investment Societies,Insurance,NaN,NaN,5.0,2.0,NaN,NaN,NaN,0


In [8]:
# dftrain ve dftesti veri çerçevelerini birleştirme
df = pd.concat([dftrain, dftest], ignore_index=True)

In [9]:
df.shape

(55414, 21)

In [10]:
df.isnull().sum()

ID                      0
Job                     0
Jobs_Group              0
Profile             35168
Remote              32141
Company                19
Location               21
City                 6449
State                5231
Frecuency_Salary        0
Mean_Salary             0
Skills                  0
Sector              11949
Sector_Group        11949
Revenue             30340
Employee            21217
Company_Score       14590
Reviews             14590
Director            34446
Director_Score      36388
URL                 26552
dtype: int64

In [11]:
columns_with_nan = ['Profile', 'Remote', 'Company', 'Location', 'City', 'State', 'Sector', 'Sector_Group', 'Revenue', 'Employee', 'Company_Score', 'Reviews', 'Director', 'Director_Score', 'URL']

for column in columns_with_nan:
    df[column].dropna(inplace=True)

In [12]:
df = df.fillna(0)

In [13]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 55414 entries, 0 to 55413
Data columns (total 21 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   ID                55414 non-null  object 
 1   Job               55414 non-null  object 
 2   Jobs_Group        55414 non-null  object 
 3   Profile           55414 non-null  object 
 4   Remote            55414 non-null  object 
 5   Company           55414 non-null  object 
 6   Location          55414 non-null  object 
 7   City              55414 non-null  object 
 8   State             55414 non-null  object 
 9   Frecuency_Salary  55414 non-null  object 
 10  Mean_Salary       55414 non-null  float64
 11  Skills            55414 non-null  object 
 12  Sector            55414 non-null  object 
 13  Sector_Group      55414 non-null  object 
 14  Revenue           55414 non-null  object 
 15  Employee          55414 non-null  object 
 16  Company_Score     55414 non-null  float6

In [14]:
df.drop(columns=['ID'], inplace=True)

In [15]:
df['Profile'] = df['Profile'].replace({'Senior': 3, 'Lead': 2, 'Junior': 1}).astype(int)
df['Remote'] = df['Remote'].replace({'Remote': 0, 'Hybrid': 1}).astype(int)
df['Frecuency_Salary'] = df['Frecuency_Salary'].replace({'year': 1, 'hour': 0, 'month': 2, 'day': 3, 'week': 4}).astype(int)

In [16]:
# Sözlük oluştur
revenue_mapping = {
    'XXXL': 1,
    'XL': 2,
    'M': 3,
    'XXL': 4,
    'XS': 5,
    'S': 6,
    'L': 7,
    'XXS': 8,
    'XXXS': 9,
}

# 'Revenue' sütunundaki değerleri replace ile sayısal değerlere dönüştürme
df['Revenue'] = df['Revenue'].replace(revenue_mapping)
# 'Revenue' sütununu int veri tipine çevirme
df['Revenue'] = df['Revenue'].astype(int)

In [17]:
# job groups sutunu job sutunu ile benzer ifadelere sahip olduğundan silebilirz
df = df.drop('Job', axis=1)
# cıtyden ötürü location ve state siliyoruz
df = df.drop('Location', axis=1)
df = df.drop('State', axis=1)
# sector grouptan ötürü sector u siliyoruz
df = df.drop('Sector', axis=1)
# Employee revenue ile benzer silelim
df = df.drop('Employee', axis=1)
# Director a gerek olmadığını düşünüyoruz
df = df.drop('Director', axis=1)
# URL de gerek yok
df = df.drop('URL', axis=1)

In [18]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 55414 entries, 0 to 55413
Data columns (total 13 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   Jobs_Group        55414 non-null  object 
 1   Profile           55414 non-null  int64  
 2   Remote            55414 non-null  int64  
 3   Company           55414 non-null  object 
 4   City              55414 non-null  object 
 5   Frecuency_Salary  55414 non-null  int64  
 6   Mean_Salary       55414 non-null  float64
 7   Skills            55414 non-null  object 
 8   Sector_Group      55414 non-null  object 
 9   Revenue           55414 non-null  int64  
 10  Company_Score     55414 non-null  float64
 11  Reviews           55414 non-null  float64
 12  Director_Score    55414 non-null  float64
dtypes: float64(4), int64(4), object(5)
memory usage: 5.5+ MB


In [19]:
df_get = pd.get_dummies(df, columns=['Jobs_Group', 'Company', 'City', 'Skills', 'Sector_Group'])

In [20]:
df_get.shape

(55414, 38454)

In [21]:
df_get.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 55414 entries, 0 to 55413
Columns: 38454 entries, Profile to Sector_Group_Transportation and Logistics
dtypes: bool(38446), float64(4), int64(4)
memory usage: 2.0 GB


In [22]:
# Eğitim ve Test verilerini modele vermeden ayırıyoruz
train=df_get[:33248]
test=df_get[33248:]

In [23]:
train.info

<bound method DataFrame.info of        Profile  Remote  Frecuency_Salary  Mean_Salary  Revenue  Company_Score  \
0            0       0                 0   115000.000        0            4.2   
1            0       0                 1   185000.000        0            3.5   
2            3       0                 1    84500.000        9            3.4   
3            3       1                 1   111625.000        4            3.8   
4            0       0                 0   102690.400        0            4.2   
...        ...     ...               ...          ...      ...            ...   
33243        0       0                 1    47206.495        0            3.6   
33244        0       0                 1    79741.000        0            3.3   
33245        0       0                 1   119908.000        2            4.3   
33246        0       0                 0   115000.000        0            0.0   
33247        2       0                 1   155000.000        0            3.5

In [24]:
test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 22166 entries, 33248 to 55413
Columns: 38454 entries, Profile to Sector_Group_Transportation and Logistics
dtypes: bool(38446), float64(4), int64(4)
memory usage: 814.1 MB


In [25]:
# MODELLEME: Makina Öğrenmesi
x=train.drop('Mean_Salary',axis=1)
y=train[['Mean_Salary']]

def fnc_all_regression_models(x,y):
    from sklearn.linear_model import LinearRegression
    from sklearn.linear_model import Ridge,Lasso
    from sklearn.linear_model import ElasticNet
    from sklearn.tree import ExtraTreeRegressor
    from sklearn.ensemble import GradientBoostingRegressor
    from sklearn.neighbors import KNeighborsRegressor

    from sklearn.model_selection import train_test_split

    from sklearn.metrics import mean_squared_error,r2_score,mean_absolute_error


    x_train,x_test,y_train,y_test=train_test_split(x,y,test_size=0.2,random_state=42)

    L=LinearRegression()
    R=Ridge()
    Lass=Lasso()
    E=ElasticNet()
    ExTree=ExtraTreeRegressor()
    GBR=GradientBoostingRegressor()
    KN=KNeighborsRegressor()

    algos=[L,R,Lass,E,ExTree,GBR,KN]
    algo_names=['LinearRegression','Ridge','Lasso','ElasticNet','ExtraTreeRegressor','GradientBoostingRegressor','KNeighborsRegressor']
    r_squared=[]
    rmse=[]
    mae=[]

    result=pd.DataFrame(columns=['R_Squared','RMSE','MAE'],index=algo_names)

    for item in algos:
        item.fit(x_train,y_train)
        item.predict(x_test)
        r_squared.append(r2_score(y_test,item.predict(x_test)))
        rmse.append((mean_squared_error(y_test,item.predict(x_test)))**.5)
        mae.append(mean_absolute_error(y_test,item.predict(x_test)))

    result.R_Squared=r_squared
    result.RMSE=rmse
    result.MAE=mae

    return result.sort_values('R_Squared',ascending=False)

fnc_all_regression_models(x,y)

In [26]:
!pip install pycaret

  Obtaining dependency information for pycaret from https://files.pythonhosted.org/packages/eb/43/ec8d59a663e0a1a67196b404ec38ccb0051708bad74a48c80d96c61dd0e5/pycaret-3.2.0-py3-none-any.whl.metadata
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.9/79.9 MB 15.2 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.8/11.8 MB 72.7 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.1/12.1 MB 72.2 MB/s eta 0:00:0000:0100:01
  Obtaining dependency information for plotly-resampler>=0.8.3.1 from https://files.pythonhosted.org/packages/08/1d/87d4ed45c26226630bcb0a205ff006c00645cc68977e22c0f6f16a7f5d2b/plotly_resampler-0.9.1-py3-none-any.whl.metadata
  Obtaining dependency information for pmdarima!=1.8.1,<3.0.0,>=1.8.0 from https://files.pythonhosted.org/packages/ec/2b/e7d18360d56396b62781ba4616527af49244d4bed51f0780646fa3953cc8/pmdarima-2.0.4-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.manylinux_2_28_x86_64.whl.metadata
     ━━━━━━━━━━

In [28]:
from pycaret.regression import*

In [29]:
setup(train,target='Mean_Salary')

,Description,Value
0,Session id,6174
1,Target,Mean_Salary
2,Target type,Regression
3,Original data shape,"(33248, 38454)"
4,Transformed data shape,"(33248, 38454)"
5,Transformed train set shape,"(23273, 38454)"
6,Transformed test set shape,"(9975, 38454)"
7,Numeric features,7
8,Preprocess,True
9,Imputation type,simple


In [ ]:
best_model=compare_models()

,,
,,
Initiated,. . . . . . . . . . . . . . . . . .,19:40:11
Status,. . . . . . . . . . . . . . . . . .,Fitting 10 Folds
Estimator,. . . . . . . . . . . . . . . . . .,Linear Regression


,Model,MAE,MSE,RMSE,R2,RMSLE,MAPE,TT (Sec)


Processing:   0%|          | 0/85 [00:00<?, ?it/s]

In [ ]:
# DERİN ÖĞRENME
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from sklearn.model_selection import train_test_split
from sklearn.metrics import r2_score
from sklearn.metrics import mean_squared_error
import matplotlib.pyplot as plt

In [ ]:
x=train.drop('Mean_Salary',axis=1)
y=train[['Mean_Salary']]

In [ ]:
x_train,x_test,y_train,y_test=train_test_split(x,y, train_size=0.80,random_state=42)

In [ ]:
model=Sequential()
model.add(Dense(80,activation='relu'))
model.add(Dense(120,activation='relu'))
model.add(Dense(64,activation='relu'))
model.add(Dense(32,activation='relu'))
model.add(Dense(16,activation='relu'))
model.add(Dense(8,activation='relu'))
model.add(Dense(1)) #regression
model.compile(loss='mse',optimizer='adam',metrics='mse')

In [ ]:
history=model.fit(x_train,y_train,validation_data=(x_test,y_test),epochs=1500,batch_size=128,verbose=1)

In [ ]:
model.summary()

In [ ]:
model.evaluate(x_test,y_test)

In [ ]:
tahmin=model.predict(x_test)

In [ ]:
r2_score(tahmin,y_test)

In [ ]:
loss_df=pd.DataFrame(history.history)
loss_df.plot(figsize=(12,8)) #eğitimin grafiği

In [ ]:
(mean_squared_error(y_test,tahmin))**0.5

In [ ]:
#tahmin ettiğimiz değer ile gerçek değerin grafiği
fig=plt.figure(figsize=(10,5))
plt.scatter(y_test,tahmin)
plt.plot(y_test,y_test,'r')